# Phân Tích Phân Khúc Khách Hàng (RFM Analysis) - Chi Tiết

Notebook này phân tích sâu các phân khúc khách hàng đã được tính toán. Mục tiêu là hiểu rõ hành vi của từng nhóm để đưa ra chiến lược kinh doanh phù hợp.

### Mục lục:
1. **Tải dữ liệu**: Đọc dữ liệu đã xử lý.
2. **Tổng quan dữ liệu**: Kiểm tra phân phối cơ bản.
3. **Phân phối R-F-M**: Xem biểu đồ Histograms cho từng chỉ số.
4. **Phân tích Phân khúc**: Trực quan hóa số lượng và giá trị của từng nhóm.
5. **Biểu đồ Treemap**: Cái nhìn tổng thể về giá trị khách hàng.
6. **Chiến lược hành động**: Đề xuất cho từng nhóm.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

PROJECT_ROOT = Path('../')
DATA_PATH = PROJECT_ROOT / 'data/processed/customer_segments.parquet'

print("Libraries loaded.")

## 1. Tải Dữ Liệu

In [ ]:
try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Loaded {len(df)} customers.")
    display(df.head())
except FileNotFoundError:
    print("File not found. Please run 'python -m src.features.rfm' first.")

## 2. Thống Kê Mô Tả
Xem các chỉ số thống kê cơ bản của Recency, Frequency, Monetary.

In [ ]:
cols = ['Recency', 'Frequency', 'Monetary']
display(df[cols].describe())

## 3. Phân Phối R-F-M (Histograms)
Hiểu sự phân bố dữ liệu giúp phát hiện các điểm bất thường hoặc xu hướng chung.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(df['Recency'], bins=50, kde=True, ax=axes[0])
axes[0].set_title('Phân phối Recency (Ngày)')

sns.histplot(df[df['Frequency'] < 100]['Frequency'], bins=50, kde=True, ax=axes[1]) # Zoom in < 100
axes[1].set_title('Phân phối Frequency (Số đơn < 100)')

sns.histplot(df[df['Monetary'] < 10000]['Monetary'], bins=50, kde=True, ax=axes[2]) # Zoom in < 10k
axes[2].set_title('Phân phối Monetary (Doanh thu < 10k)')

plt.tight_layout()
plt.show()

## 4. Phân Tích Chi Tiết Phân Khúc

### 4.1 Số lượng khách hàng mỗi nhóm

In [ ]:
segment_counts = df['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']

fig = px.bar(segment_counts, x='Count', y='Segment', orientation='h', 
             title='Số Lượng Khách Hàng Mỗi Phân Khúc', 
             text='Count', 
             color='Segment')
fig.update_layout(showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.show()

### 4.2 Giá trị trung bình của từng nhóm (Average Monetary)
Nhóm nào mang lại giá trị đơn hàng cao nhất?

In [ ]:
avg_monetary = df.groupby('Segment')['Monetary'].mean().reset_index()

fig = px.bar(avg_monetary, x='Monetary', y='Segment', orientation='h', 
             title='Giá Trị Chi Tiêu Trung Bình Mỗi Khách Hàng', 
             color='Segment')
fig.update_layout(showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.show()

## 5. Trực Quan Hóa Treemap
Biểu đồ Treemap giúp nhìn thấy tỷ trọng của từng nhóm so với tổng thể.

In [ ]:
fig = px.treemap(df, path=['Segment'], values='Monetary',
                 title='Tỷ Trọng Doanh Thu Theo Phân Khúc',
                 color='Monetary',
                 color_continuous_scale='RdBu')
fig.show()

## 6. Scatter Plot 3D (R-F-M)
Nhìn không gian 3 chiều của dữ liệu khách hàng.

In [ ]:
fig = px.scatter_3d(df.sample(1000), x='Recency', y='Frequency', z='Monetary',
                    color='Segment', opacity=0.7,
                    title='3D View: Recency - Frequency - Monetary (Sample 1000 customers)')
fig.show()

## 7. Kết Luận & Hành Động (Actionable Insights)

| Phân Khúc | Đặc Điểm | Hành Động Đề Xuất |
| :--- | :--- | :--- |
| **Champions (VIP)** | Mua gần đây, thường xuyên, chi nhiều. | Ưu tiên hàng đầu. Tặng quà, tiếp thị sản phẩm mới sớm nhất. Không cần giảm giá. |
| **Loyal Customers** | Mua thường xuyên. | Upsell sản phẩm giá trị cao hơn. Mời tham gia chương trình khách hàng thân thiết. |
| **Potential Loyalist** | Mua gần đây, chi tiêu trung bình. | Cần khuyến khích mua thêm để trở thành Loyal. Gửi mã giảm giá cho lần mua sau. |
| **At Risk** | Đã từng mua nhiều nhưng lâu không quay lại. | Rất quan trọng! Cần gọi điện hoặc gửi email cá nhân hóa để tìm hiểu lý do và kéo họ lại. |
| **Hibernating** | Lâu không mua, chi tiêu ít. | Tiết kiệm ngân sách marketing cho nhóm này. Chỉ gửi thông tin khi có đợt giảm giá lớn. |